In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from math import log

In [19]:
k1 = pd.read_csv('./Result/oof_test_1.csv')
k2 = pd.read_csv('./Result/oof_test_2.csv')
k3 = pd.read_csv('./Result/oof_test_3.csv')
k4 = pd.read_csv('./Result/oof_test_4.csv')
k5 = pd.read_csv('./Result/oof_test_5.csv')

k = pd.concat([k1,k2,k3,k4,k5])

reverse_hla = LabelEncoder()
reverse_hla.classes_ = np.load('./Data/hla_encoder_classes.npy')
  
k['hla'] = reverse_hla.inverse_transform(k['hla'])

In [23]:
k['aff'] = k['aff'].apply(lambda x: 1 if x >= (1-log (500)/log(50000)) else 0)

In [30]:
alpha = k.groupby(['hla', 'peptide'])['oof_preds'].transform(max) == k['oof_preds']
k = k[alpha]

In [42]:
Scoring = {}
for h in k['hla'].unique():
  tmp = k[k['hla'] == h]
  try:
    score = roc_auc_score(tmp['aff'], tmp['oof_preds'])
    Scoring[h] = [score]
  except:
    Scoring[h] = [np.nan]

In [43]:
final_table = pd.DataFrame(Scoring).T
#final_table